In [ ]:
import re
import csv
from pathlib import Path
from PyPDF2 import PdfReader

pdf_path = Path(r"C:/BD/Dados.pdf")
out_path = Path(r"C:/BD/Pontos.csv")

re_rodovia = re.compile(r"Rodovia:\s*([A-Z]+\d+)")
re_codigo = re.compile(r"C[oó]digo do Passivo:\s*([^\n\r]+)")
re_km      = re.compile(r"Km:(\d+\+\d+)")
re_coord   = re.compile(
    r"Coordenada:\s*(\d{2}[NS])\s*([\d\.,]+)\s*m\s*E\s*([\d\.,]+)\s*m\s*[NS]",
    re.I
)

rows = []
reader = PdfReader(str(pdf_path))
for page in reader.pages:
    text = page.extract_text()

    rodovias = re_rodovia.findall(text)
    codigos  = re_codigo.findall(text)
    kms      = re_km.findall(text)
    coords   = re_coord.findall(text)

    # combina só o que tem tudo
    n = min(len(rodovias), len(codigos), len(kms), len(coords))
    for i in range(n):
        rodovia = rodovias[i]
        codigo  = codigos[i]
        km      = kms[i]

        zona, east_raw, north_raw = coords[i]

        # limpa pontos de milhar e troca vírgula -> ponto
        east  = east_raw.replace('.', '').replace(',', '.').strip()
        north = north_raw.replace('.', '').replace(',', '.').strip()

        # AGORA: X = Easting, Y = Northing  
        rows.append([rodovia, codigo, km, zona, east, north])

# grava CSV
with open(out_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow([
        "Rodovia", "Código do Passivo", "Km",
        "Zona", "X_Easting", "Y_Northing"
    ])
    writer.writerows(rows)

print(f"Gerado {out_path} com {len(rows)} linhas")